In [25]:
import pandas as pd
import numpy as np

In [35]:
def get_potential_outcomes(df):
    
    # Initialize y0 and y1
    df['y0'] = np.nan
    df['y1'] = np.nan
    
    # Get observed outcomes
    df.loc[df.W==0, 'y0'] = df.loc[df.W==0, 'y_true']
    df.loc[df.W==1, 'y1'] = df.loc[df.W==1, 'y_true']
    
    # Get unobserved outcomes
    df.loc[df.W==0, 'y1'] = df.loc[df.W==0, 'y0'] + df.loc[df.W==0, 'tau_true']
    df.loc[df.W==1, 'y0'] = df.loc[df.W==1, 'y1'] - df.loc[df.W==1, 'tau_true']
    
    return df

In [37]:
df_T = pd.read_parquet('preds/T_simA_iw_g_rfc_default_test_preds.parquet')
df_T.head()

# transform y_true into y0/y1 with tau preds 
df_T = get_potential_outcomes(df_T)
df_T

# rf
# compare y0 to y0_preds
# compare y1_preds to Y1
# compare tau_preds to tau_true
# color code all of above by W 

# iw 
# pscore_preds vs pscore true (KELSEY GET FROM ORIG FILES)

,y1_preds,y0_preds,tau_preds,tau_true,y_true,W,pscore_true,y0,y1
0,-2.844000,-4.003843,1.159842,0.0,-6.016433,0,0.01,-6.016433,-6.016433
1,5.529967,2.251538,3.278429,0.0,2.303158,0,0.01,2.303158,2.303158
2,-7.822841,-8.189758,0.366917,0.0,-7.733460,0,0.01,-7.733460,-7.733460
3,16.532348,14.335522,2.196826,0.0,11.494454,0,0.01,11.494454,11.494454
4,20.408202,17.734280,2.673922,0.0,17.661586,0,0.01,17.661586,17.661586
...,...,...,...,...,...,...,...,...,...
99995,36.739040,37.156092,-0.417052,0.0,35.810106,0,0.01,35.810106,35.810106
99996,9.494531,7.720591,1.773940,0.0,4.897084,0,0.01,4.897084,4.897084
99997,-12.942568,-16.938051,3.995483,0.0,-14.333449,0,0.01,-14.333449,-14.333449
99998,-0.077888,-1.891000,1.813112,0.0,-2.126215,0,0.01,-2.126215,-2.126215


In [5]:
df_S = pd.read_parquet('preds/S_simA_iw_g_rfc_default_test_preds.parquet')
df_S.head()

# rf and iw
# compare y0 to y0_preds
# compare y1_preds to Y1
# compare tau_preds to tau_true
# color code all of above by W 

,y1_preds,y0_preds,tau_preds,tau_true,y_true,W,pscore_true
0,-0.381054,-4.034165,3.653111,0.0,-6.016433,0,0.01
1,5.900143,2.247033,3.653111,0.0,2.303158,0,0.01
2,-4.572827,-8.225938,3.653111,0.0,-7.733460,0,0.01
3,17.966926,14.313816,3.653111,0.0,11.494454,0,0.01
4,21.381651,17.728541,3.653111,0.0,17.661586,0,0.01


In [6]:
df_X_test = pd.read_parquet('preds/X_simA_iw_g_rfc_default_test_preds.parquet')
df_X_test.head()

# rf
# tau_preds vs tau_true
# pscore_preds vs pscore_true
# color code by W

# iw
# same as above

,tau_preds,pscore_preds,tau_true,y_true,W,pscore_true
0,1.139436,0.09,0.0,-6.016433,0,0.01
1,3.143615,0.01,0.0,2.303158,0,0.01
2,0.412934,0.01,0.0,-7.733460,0,0.01
3,2.118066,0.01,0.0,11.494454,0,0.01
4,2.474081,0.04,0.0,17.661586,0,0.01


In [8]:
df_X_train = pd.read_parquet('preds/X_simA_iw_g_rfc_default_train_preds.parquet')
df_X_train.head()


# rf
# y0_treat_preds vs y_0 for ppl who have W =1  (KELSEY GET FROM ORIG FILES)
# y1_control_preds vs y1 for ppl who have W =0 (KELSEY GET FROM ORIG FILES)
# color code by W

# iw
# same as above
# pscore_preds vs pscore true (KELSEY GET FROM ORIG FILES)

,y0_treat_preds,y,W,pscore_preds,y1_control_preds
0,7.453906,NaN,1,0.65,NaN
1,20.519630,NaN,1,0.65,NaN
2,14.224256,NaN,1,0.71,NaN
3,8.052662,NaN,1,0.64,NaN
4,2.784312,NaN,1,0.64,NaN


In [9]:
df_X_train.tail()

,y0_treat_preds,y,W,pscore_preds,y1_control_preds
299995,NaN,22.616128,0,0.0,-19.441320
299996,NaN,18.219507,0,0.0,10.882998
299997,NaN,22.920906,0,0.0,-14.150874
299998,NaN,-13.287223,0,0.0,17.879945
299999,NaN,3.921537,0,0.0,19.063774
